In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ReadData as rd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression  # this is your ALGORITHM
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll import scope

import mlflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('taxi')
df = rd.read_data()

categorical = ['PULocationID', 'DOLocationID']
target = 'duration'
df[categorical] = df[categorical].astype(str)

df['pu_do'] = df['PULocationID'] + '_' + df['DOLocationID']

df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

numerical = ['trip_distance']
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
test_dicts = df_test[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df_train['duration'].values
X_test = dv.transform(test_dicts)
y_test = df_test['duration'].values


train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_test, label=y_test)


def train_model(params):
    with mlflow.start_run():
        mlflow.log_params(params)
        booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, 'validation')],
            early_stopping_rounds = 50,
        )
        y_pred = booster.predict(X_train)
        rmse = root_mean_squared_error(y_train, y_pred)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'reg_alpha': hp.uniform('reg_alpha', 1, 5),
    'reg_lambda': hp.uniform('reg_lambda', 1, 6),
    'min_child_weight': hp.uniform('min_child_weight', 1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}
best_result = fmin(
    fn=train_model,
    space=search_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=Trials(),
)

